In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import  train_test_split

In [2]:
df = pd.read_csv('/content/bank.csv')
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,deposit
0,59,admin.,married,secondary,no,2343,yes,no,unknown,5,may,1042,1,-1,0,unknown,yes
1,56,admin.,married,secondary,no,45,no,no,unknown,5,may,1467,1,-1,0,unknown,yes
2,41,technician,married,secondary,no,1270,yes,no,unknown,5,may,1389,1,-1,0,unknown,yes
3,55,services,married,secondary,no,2476,yes,no,unknown,5,may,579,1,-1,0,unknown,yes
4,54,admin.,married,tertiary,no,184,no,no,unknown,5,may,673,2,-1,0,unknown,yes


In [3]:
df.describe()

,age,balance,day,duration,campaign,pdays,previous
count,11162.000000,11162.000000,11162.000000,11162.000000,11162.000000,11162.000000,11162.000000
mean,41.231948,1528.538524,15.658036,371.993818,2.508421,51.330407,0.832557
std,11.913369,3225.413326,8.420740,347.128386,2.722077,108.758282,2.292007
min,18.000000,-6847.000000,1.000000,2.000000,1.000000,-1.000000,0.000000
25%,32.000000,122.000000,8.000000,138.000000,1.000000,-1.000000,0.000000
50%,39.000000,550.000000,15.000000,255.000000,2.000000,-1.000000,0.000000
75%,49.000000,1708.000000,22.000000,496.000000,3.000000,20.750000,1.000000
max,95.000000,81204.000000,31.000000,3881.000000,63.000000,854.000000,58.000000


In [4]:
df2 = df [['age', 'balance', 'day', 'duration', 'campaign', 'pdays']].apply(lambda x: (x - x.min()) / (x.max() - x.min()))
df2 ['deposit'] = df ['deposit']
df2.head ()

,age,balance,day,duration,campaign,pdays,deposit
0,0.532468,0.104371,0.133333,0.268110,0.000000,0.0,yes
1,0.493506,0.078273,0.133333,0.377675,0.000000,0.0,yes
2,0.298701,0.092185,0.133333,0.357566,0.000000,0.0,yes
3,0.480519,0.105882,0.133333,0.148750,0.000000,0.0,yes
4,0.467532,0.079851,0.133333,0.172983,0.016129,0.0,yes


##Start of Neural network function

In [5]:
#Activation functions

def sigmoid(x):  
    return 1/(1 + np.exp(-x))

def sigmoid_derivative(x):  
    return sigmoid(x)*(1 - sigmoid(x))

def relu(x):
    return np.maximum(0,x)

def relu_derivative(x):
    x[x <= 0] = 0
    x[x > 0] = 1
    return x   


### Back Propogation

In [6]:
def back_prop(inputs, hw, ow, hb, ob, alpha, zh, ah, zo, diff):
    
    # output layer weights cost function
    dcost_dao = diff
    dao_dzo = sigmoid_derivative(zo) 
    dzo_dwo = ah
    dcost_wo = np.dot(dzo_dwo.T, dcost_dao * dao_dzo)
    
    # hidden layer weights cost function
    dcost_dzo = dcost_dao * dao_dzo
    dzo_dah = ow
    dcost_dah = np.dot(dcost_dzo , dzo_dah.T)
    dah_dzh = relu_derivative(zh) 
    dzh_dwh = inputs
    dcost_wh = np.dot(dzh_dwh.T, dah_dzh * dcost_dah)
    
    # update weights
    hw -= alpha * dcost_wh
    ow -= alpha * dcost_wo
    
    # update biases
    for d in dcost_wh:
        hb -= alpha * d
    
    for d in dcost_wo:
        ob -= alpha * d

### Feed Forward and Model Training


In [7]:
def mlp(inputs, outputs, hidden_layer_width, alpha, epochs):
    
    d = dict()
    np.random.seed(42)

    hw = np.random.rand(len(inputs[0]), hidden_layer_width)
    ow = np.random.rand(hidden_layer_width, 1)  

    # intialize biases for hidden and output layers to zeros
    hb = np.zeros((1, hidden_layer_width))
    ob = np.zeros((1, 1))

    # learning rate
    learning_rate = alpha

    for epoch in range(1, epochs + 1):
        # hidden layer
        zh = np.dot(inputs, hw) + hb
        ah = relu(zh)
    
        # output layer
        zo = np.dot(ah, ow) + ob
        ao = sigmoid(zo)
    
        # calculate mean squared error
        diff = ao - outputs
        mse = (np.square(diff)).mean(axis=0)
        
        # backpropagation
        back_prop(inputs, hw, ow, hb, ob, alpha, zh, ah, zo, diff)
    
        # save the final weights and biases of the model for testing
        if epoch == epochs:
            print("After %d epochs, mean squared error is: %.2f" % (epoch,mse[0]))
            d['hidden layer weights'] = hw
            d['output layer weights'] = ow
            d['hidden layer biases'] = hb
            d['output layer biases'] = ob
            
    return d

### Model Accuracy

In [14]:
def model_accuracy(test_inputs, test_outputs, hidden_weights, output_weights, hidden_biases, output_biases):
    
    # hidden layer
    zh = np.dot(test_inputs, hidden_weights) + hidden_biases
    ah = relu(zh)
    
    # output layer
    zo = np.dot(ah, output_weights) + output_biases
    y = sigmoid(zo)
    
    preds_correct_boolean =  np.argmax(y, 1) == np.argmax(test_outputs, 1)
    correct_predictions = np.sum(preds_correct_boolean)
    accuracy = correct_predictions / len(test_outputs)
    print("Accuracy is: %f" % accuracy)

###Inputs and Outputs

In [9]:

X = df2.iloc[:, 0:5].values
y = df2.iloc[:, 6].values
y = pd.factorize(y)[0]
X, y

# 75-25 split for training and testing sets
split = round(.75 * len(X))

X_train = np.array(X[:split])
X_test = np.array(X[split:])

y_train = np.array(y[:split])
y_test = np.array(y[split:])

y_train = y_train.reshape(len(y_train),1)
y_test = y_test.reshape(len(y_test),1)

In [10]:
model = mlp(inputs=X_train, outputs=y_train, hidden_layer_width=40, alpha=0.05, epochs=10000)

After 10000 epochs, mean squared error is: 0.63


In [13]:
hw = model['hidden layer weights']
ow = model['output layer weights']
hb = model['hidden layer biases']
ob = model['output layer biases']

model_accuracy(test_inputs=X_test, test_outputs=y_test, hidden_weights=hw, output_weights=ow, 
               hidden_biases=hb, output_biases=ob)


Accuracy is: 0.790000
